In [54]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableMap
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_groq import ChatGroq
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List,Optional,Dict
import os
load_dotenv()

True

## Lang chain Expression Model 

In [7]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")

In [40]:
prompt = ChatPromptTemplate([
    "tell me a joke on {topic}"
])
model= ChatGroq(model="gemma2-9b-it")
output_praser = StrOutputParser()

In [41]:
chain1 = prompt|model|output_praser

In [42]:
chain1.invoke({"topic":"cat"})

"Why don't cats play poker in the jungle? \n\nToo many cheetahs! 😹  \n"

In [18]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [23]:
vectorstore = DocArrayInMemorySearch.from_texts(["harrison worked at khensho","bears like to eat honey"],embedding=OpenAIEmbeddings())
retriver = vectorstore.as_retriever()

In [24]:
retriver.get_relevant_documents("Where did harrision work")

/var/folders/1s/v1ppljh905v67_hwc79hwjq00000gt/T/ipykernel_44331/633080556.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriver.get_relevant_documents("Where did harrision work")


[Document(metadata={}, page_content='harrison worked at khensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [35]:
template = """Answere the Question based on the following context:{context}
Question:{question}"""
prompt = ChatPromptTemplate.from_template(template=template)

In [61]:
openai_model = ChatOpenAI(model="gpt-4o-mini")

In [37]:
chain = RunnableMap({
    "context": lambda x:retriver.get_relevant_documents(x['question']),
    "question":lambda x:x['question']
})|prompt|openai_model|output_praser

In [38]:
chain.invoke({"question":"where did harrison work"})

'Harrison worked at Khensho.'

In [39]:
#model binds and fallback concept 
#inference 

In [44]:
chain1.batch([{"topic":"cat"},{"topic":"dog"}])

["Why don't cats play poker in the jungle? \n\nToo many cheetahs!  😹  \n",
 "Why do dogs run in circles? \n\nBecause it's really hard to run in squares! 🐶  \n"]

In [46]:
for t in chain1.stream({"topic":'bears'}):
    print(t)


Why
 don
'
t
 bears
 wear
 shoes
?
 



Because
 they
 have
 bear
 feet
!
  
🐻
 





In [49]:
response = await chain1.ainvoke({"topic":"bears"})

In [50]:
response

"Why don't bears wear shoes? \n\nBecause they have bear feet! 🐻 🐾  \n"

## Open AI Function calling in Langchain 

In [ ]:
## pydantic 

In [53]:
# pydantic to Open AI Function Defnition 

In [55]:
class WeatherSearch(BaseModel):
 """Call this with an airport code to get the weather at that airport"""
 airport_code :str = Field(description="Airport code to get weather near the airport")

In [56]:
weather_finction = convert_pydantic_to_openai_function(WeatherSearch)

/var/folders/1s/v1ppljh905v67_hwc79hwjq00000gt/T/ipykernel_44331/1860383405.py:1: LangChainDeprecationWarning: The function `_convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  weather_finction = convert_pydantic_to_openai_function(WeatherSearch)


In [62]:
model_with_function_bind = openai_model.bind(functions=[weather_finction])

In [68]:
model_with_function_bind_force = openai_model.bind(functions=[weather_finction],function_call={"name":"WeatherSearch"})

In [71]:
chain = model_with_function_bind_force

In [72]:
chain.invoke("Whats the weather in SF today?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 79, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_92e0377081', 'id': 'chatcmpl-BaesEhk4lgcRBvnQrhjBfLgqCWvob', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--52b7027a-d5ce-45ea-bbfe-2ce253f5771f-0', usage_metadata={'input_tokens': 79, 'output_tokens': 7, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [74]:
prompt= ChatPromptTemplate({
    ("system","You are helpful assistant"),
    ("user","{input}")
})
chain = prompt|model_with_function_bind
chain.invoke({"input","Whats the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 77, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BaeuhePVTrnQ1K6dntATR5rGmS8sK', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--e68d5a0c-9adb-470a-90a5-b885d229b8db-0', usage_metadata={'input_tokens': 77, 'output_tokens': 17, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Taging and extraction 

In [ ]:
# real word example 

## tools and routing 
